<a href="https://colab.research.google.com/github/isdeniz/msc-2024/blob/main/llama3_msc_summary_and_preds_with_70b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [2]:
!python3.10 -m pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 391.5/391.5 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 13.7 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [3]:
!python3.10 -m pip install langchain-core

In [4]:
!python3.10 -m pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.4 MB/s eta 0:00:00


In [5]:
from langchain_community.llms import Ollama
llm = Ollama(model = "llama3:70b", num_ctx=8192, temperature=0)

In [6]:
pip install tqdm

In [7]:
import pandas as pd
from tqdm import tqdm
import numpy as np

In [8]:
test_sample = pd.read_pickle('data/test_sample_llama_5000_to_be_summarized_github.pkl')
print('Number of issues: ', test_sample.shape)

Number of issues:  (28, 26)


In [9]:
test_sample.head()

,index,original_issue_title,preprocessed_title,original_issue_body,preprocessed_body,original_label,text,issue_label,labels,roberta_prediction,...,llama3_8b_understandability,llama3_8b_understandability_corrected,round2_8b,llama3_70b_classification,llama3_70b_classification_corrected,llama3_70b_confidence,llama3_70b_confidence_corrected,llama3_70b_understandability,llama3_70b_understandability_corrected,round2_70b
68,3007,Jpa namedQuery for different data types error,jpa namedquery for different data types error,``` java\n@Entity\n@Data\n@Cacheable\n@Table(n...,java entity data cacheable table name user pub...,['for: stackoverflow'],time 2016-01-15T09:05:31Z author NONE repo spr...,question,2,question,...,,,True,,,,,,,True
85,9537,Druid query returns 500 error with javax.servl...,druid query returns 500 error with javax.servl...,### Affected Version\r\n0.22.1\r\n\r\n### Desc...,affected version 0.22.1 description after upgr...,"['Bug', 'Area - SQL']",time 2022-09-15T07:08:19Z author CONTRIBUTOR r...,bug,0,bug,...,,,True,,,,,,,True
132,3755,WLS 12c spring-boot war deploy fail,wls 12c spring boot war deploy fail,>>The ResourceConfig instance does not contain...,the resourceconfig instance does not contain a...,['for: stackoverflow'],time 2017-02-23T09:28:08Z author NONE repo spr...,question,2,question,...,,90,False,,,,,,,True
241,3767,My Game Crashes,my game crashes,**Platform**: *Windows*\r\n\r\n**Build**: *The...,platform windows build the build number 104.10...,['bug'],time 2020-06-24T11:29:22Z author NONE repo Anu...,bug,0,bug,...,,,True,,,,,,,True
893,10196,[Bug] shadow topic replicator may fail due to ...,bug shadow topic replicator may fail due to re...,### Search before asking\r\n\r\n- [X] I search...,search before asking x i searched in the issue...,['type/bug'],time 2022-12-04T10:18:10Z author CONTRIBUTOR r...,bug,0,bug,...,,,True,,,,,,,True


Run for all issues :)

In [10]:
with tqdm(total=test_sample.shape[0], ncols=100) as pbar:
    for idx in test_sample.index:
        try:
            test_sample.loc[idx, 'llama3_70b_body_summary'] = llm.invoke(f"""Summarize the following but do not add your own recommendations or interpretations: {test_sample.loc[idx, 'original_issue_body']}""")
            pbar.update(1)
        except Exception as e:
            print(f'Exception at index {idx}: {e}')
            continue

print(f'Summarization of {test_sample.shape[0]} issues is complete')

100%|███████████████████████████████████████████████████████████████| 28/28 [30:22<00:00, 65.09s/it]

Summarization of 28 issues is complete


Check summaries

In [11]:
test_sample

,index,original_issue_title,preprocessed_title,original_issue_body,preprocessed_body,original_label,text,issue_label,labels,roberta_prediction,...,llama3_8b_understandability_corrected,round2_8b,llama3_70b_classification,llama3_70b_classification_corrected,llama3_70b_confidence,llama3_70b_confidence_corrected,llama3_70b_understandability,llama3_70b_understandability_corrected,round2_70b,llama3_70b_body_summary
68,3007,Jpa namedQuery for different data types error,jpa namedquery for different data types error,``` java\n@Entity\n@Data\n@Cacheable\n@Table(n...,java entity data cacheable table name user pub...,['for: stackoverflow'],time 2016-01-15T09:05:31Z author NONE repo spr...,question,2,question,...,,True,,,,,,,True,This is a list of Java libraries and their cor...
85,9537,Druid query returns 500 error with javax.servl...,druid query returns 500 error with javax.servl...,### Affected Version\r\n0.22.1\r\n\r\n### Desc...,affected version 0.22.1 description after upgr...,"['Bug', 'Area - SQL']",time 2022-09-15T07:08:19Z author CONTRIBUTOR r...,bug,0,bug,...,,True,,,,,,,True,Here is a summary of the error:\n\n* The error...
132,3755,WLS 12c spring-boot war deploy fail,wls 12c spring boot war deploy fail,>>The ResourceConfig instance does not contain...,the resourceconfig instance does not contain a...,['for: stackoverflow'],time 2017-02-23T09:28:08Z author NONE repo spr...,question,2,question,...,90,False,,,,,,,True,"This is a Java application log output, specifi..."
241,3767,My Game Crashes,my game crashes,**Platform**: *Windows*\r\n\r\n**Build**: *The...,platform windows build the build number 104.10...,['bug'],time 2020-06-24T11:29:22Z author NONE repo Anu...,bug,0,bug,...,,True,,,,,,,True,"This appears to be a Java crash log, which pro..."
893,10196,[Bug] shadow topic replicator may fail due to ...,bug shadow topic replicator may fail due to re...,### Search before asking\r\n\r\n- [X] I search...,search before asking x i searched in the issue...,['type/bug'],time 2022-12-04T10:18:10Z author CONTRIBUTOR r...,bug,0,bug,...,,True,,,,,,,True,This is a log output from a test case in Apach...
1313,7907,default datasource page limit problem,default datasource page limit problem,version：4.0.0.M1-SNAPSHOT\r\nbranch：dev \r\nor...,version4.0.0.m1 snapshot branchdev orm spring ...,['type: bug'],time 2019-01-07T09:56:25Z author NONE repo apa...,bug,0,bug,...,,True,,,,,,,True,It looks like you're experiencing an issue wit...
1323,3577,play2.x plugin can't find end point,play2.x plugin can't find end point,Please answer these questions before submittin...,please answer these questions before submittin...,['question'],time 2019-10-23T08:59:18Z author NONE repo apa...,question,2,question,...,,True,,,,,,,True,It seems like you're experiencing issues with ...
1424,8731,Simulate index fails when index name already e...,simulate index fails when index name already e...,### Elasticsearch Version\n\n8.2.0\n\n### Inst...,elasticsearch version 8.2.0 installed plugins ...,"['>bug', ':Data Management/Indices APIs']",time 2022-02-23T00:01:01Z author CONTRIBUTOR r...,bug,0,bug,...,,True,,,,,,,True,This is a Java stack trace from an Elasticsear...
1557,9111,[🐛 Bug]: remote selenium fails to respond to s...,bug remote selenium fails to respond to start ...,### What happened?\r\n\r\nDiscovered in the[ J...,what happened? discovered in the jenkins ath s...,"['R-awaiting answer', 'I-defect']",time 2023-07-26T11:55:47Z author NONE repo Sel...,bug,0,bug,...,,True,,,,,,,True,This appears to be a thread dump from a Java a...
1805,4398,[Bug] Failed to start OAP。-version 9.2.0,bug failed to start oap version 9.2.0,### Search before asking\n\n- [X] I had search...,search before asking x i had searched in the i...,['bug'],time 2023-06-19T08:59:28Z author NONE repo apa...,bug,0,bug,...,,True,,,,,,,True,"This is an error log from Apache SkyWalking, a..."


In [12]:
test_sample.loc[68,'llama3_70b_body_summary']

"This is a list of Java libraries and their corresponding versions, along with their file paths on the local machine. The list appears to be generated by a build tool such as Gradle or Maven.\n\nHere's a breakdown of the information:\n\n* Each line represents a single library or dependency.\n* The format is `groupId:artifactId:version`, where:\n\t+ `groupId` is the unique identifier for the organization or group that maintains the library (e.g., `org.springframework.cloud`).\n\t+ `artifactId` is the unique identifier for the specific library (e.g., `spring-cloud-stream-binder-rabbit`).\n\t+ `version` is the version number of the library (e.g., `1.0.0.BUILD-SNAPSHOT`).\n* The file path following each dependency specifies the location of the corresponding JAR file on the local machine.\n\nThe list includes a wide range of libraries, including:\n\n* Spring Cloud and Spring Boot dependencies\n* Netflix OSS libraries (e.g., Ribbon, Eureka)\n* Apache Tomcat and HTTPComponents libraries\n* Ja

In [13]:
test_sample.loc[68,'llama3_70b_body_summary_check'] = 'OK'

In [14]:
test_sample.loc[85,'llama3_70b_body_summary']

"Here is a summary of the error:\n\n* The error occurs when there are attributes in IN and/or NOT IN clauses, and the output result is empty.\n* The LATEST(expr, maxBytesPerString) clause with 2 arguments is causing the issue, specifically the _maxBytesPerString_ argument.\n* Without a GROUP BY clause, the query does not work on both Druid 0.19 and 0.22.\n* With a GROUP BY clause, the query works on Druid 0.19 but not on Druid 0.22.\n* The error is related to Calcite's ability to parse empty result sets, specifically in the `RexLiteral.findValue(RexNode node)` method.\n\nThe stack trace shows that the error originates from the `RexLiteral` class in Calcite, and propagates through various layers of Druid's SQL processing pipeline."

In [15]:
test_sample.loc[85,'llama3_70b_body_summary_check'] = 'OK'

In [16]:
test_sample.loc[132,'llama3_70b_body_summary']

"This is a Java application log output, specifically from a Spring Boot application running on a WebLogic server. Here's a breakdown of the log:\n\n**Initial Startup**\n\n* The application starts up and initializes various components, including the Spring Boot framework, Jersey (a JAX-RS implementation), and WebLogic.\n* The application context is created, and beans are registered.\n\n**Jersey Configuration**\n\n* Jersey is configured to scan for root resource classes in the package `com.alibaba.fastjson.support.jaxrs`.\n* No root resource classes are found, which causes an error later on.\n\n**Error: No Root Resource Classes**\n\n* An error occurs when trying to initiate the Jersey application: `The ResourceConfig instance does not contain any root resource classes.`\n* This error is caused by the lack of root resource classes in the Jersey configuration.\n* The error is propagated up the stack, causing the deployment of the application to fail.\n\n**Shutdown and Error Messages**\n\n*

In [17]:
test_sample.loc[132,'llama3_70b_body_summary_check'] = 'OK'

In [18]:
test_sample.loc[241,'llama3_70b_body_summary']

"This appears to be a Java crash log, which provides detailed information about the system, environment, and events leading up to the crash.\n\nHere's a summary of the contents:\n\n**Platform**: The platform is Windows 10 (64-bit) with an OpenJDK 64-Bit Server VM (version 1.8.0_72-b15).\n\n**Events**: There are 10 events recorded, including DEOPT (deoptimization) and Uncommon trap events.\n\n**Dynamic libraries**: A list of loaded dynamic libraries is provided, including Java runtime libraries, Windows system libraries, and third-party libraries.\n\n**VM Arguments**: The log shows that the Java command and class path were not set.\n\n**Environment Variables**: Environment variables such as PATH, USERNAME, OS, and PROCESSOR_IDENTIFIER are listed.\n\n**System Information**: System details, including CPU architecture, memory usage, and operating system version, are provided.\n\nThe log also includes a confirmation request at the end, asking the user to confirm that they have searched for 

In [19]:
test_sample.loc[241,'llama3_70b_body_summary_check'] = 'OK'

In [20]:
test_sample.loc[893,'llama3_70b_body_summary']

'This is a log output from a test case in Apache Pulsar, a distributed messaging system. The test case is named `ShadowTopicTest` and it\'s testing the consumption of messages from a shadow topic.\n\nThe log shows that the consumer is trying to read messages from the topic, but it encounters an error when reading entries at ledger 7: "No such ledger exists on Bookies - Retrying to read in 15.0 seconds". This suggests that there was an issue with the ledger storage.\n\nDespite this error, the test continues and the producer sends more messages to the topic. The consumer eventually receives some messages, but it also receives a null message, which causes a `NullPointerException` when trying to access the message ID.\n\nThe expected behavior is for the consumer to receive all 10 messages sent by the producer, but instead, it encounters an error due to the null message.\n\nIt\'s not clear from this log output what the root cause of the issue is or how to fix it. Further investigation and d

In [22]:
print(test_sample.loc[893,'original_issue_body'])

### Search before asking

- [X] I searched in the [issues](https://github.com/apache/pulsar/issues) and found nothing similar.


### Version

Master branch

### Minimal reproduce step

Based on the flaky test https://github.com/apache/pulsar/issues/18705, I modified the code snippet to reproduce the bug with a high probability:

```java
@Test
    public void testShadowTopicConsuming() throws Exception {
        String sourceTopic = newShadowSourceTopicName();
        String shadowTopic = sourceTopic + "-shadow";
        admin.topics().createNonPartitionedTopic(sourceTopic);
        admin.topics().createShadowTopic(shadowTopic, sourceTopic);
        admin.topics().setShadowTopics(sourceTopic, Lists.newArrayList(shadowTopic));
        // wait init finish
        Thread.sleep(1000 * 3);

        PersistentTopic brokerSourceTopic =
                (PersistentTopic) pulsar.getBrokerService().getTopicIfExists(sourceTopic).get().get();
        ManagedLedgerConfig managedLedgerConfig = brokerS

In [23]:
test_sample['llama3_70b_body_summary_corrected'] = test_sample['llama3_70b_body_summary']

In [24]:
test_sample.loc[893,'llama3_70b_body_summary_corrected'] = 'This is a log output from a test case in Apache Pulsar, a distributed messaging system. The test case is named `ShadowTopicTest` and it\'s testing the consumption of messages from a shadow topic.\n\nThe log shows that the consumer is trying to read messages from the topic, but it encounters an error when reading entries at ledger 7: "No such ledger exists on Bookies - Retrying to read in 15.0 seconds". This suggests that there was an issue with the ledger storage.\n\nDespite this error, the test continues and the producer sends more messages to the topic. The consumer eventually receives some messages, but it also receives a null message, which causes a `NullPointerException` when trying to access the message ID.\n\nThe expected behavior is for the consumer to receive all 10 messages sent by the producer, but instead, it encounters an error due to the null message.'

In [25]:
test_sample.loc[893,'llama3_70b_body_summary_check'] = 'last two sentences are removed'

In [26]:
test_sample.loc[1313,'llama3_70b_body_summary']

"It looks like you're experiencing an issue with pagination in your database query.\n\nThe log shows that the `limit` clause is being applied differently for each query, but the resulting data is identical. This suggests that the query is not properly paginating the results.\n\nHere are a few potential reasons for this behavior:\n\n1. **Incorrect offset calculation**: Make sure that the `offset` value is being calculated correctly based on the current page number and limit.\n2. **Missing ordering clause**: Ensure that the query includes an `ORDER BY` clause to specify the sorting order of the results. Without an ordering clause, the database may return results in an arbitrary order, which can lead to pagination issues.\n3. **Inconsistent data**: Verify that the underlying data is consistent and not changing between queries.\n\nTo troubleshoot this issue, you can try the following:\n\n1. Check the query logs to ensure that the `limit` and `offset` values are being applied correctly.\n2.

In [27]:
test_sample.loc[1313,'original_issue_body']

'version：4.0.0.M1-SNAPSHOT\r\nbranch：dev \r\norm: spring jpa\r\n\r\nnot sharding table use default datasource，but  page data is the same，for the following limit:\r\n```\r\nlimit 1,10\r\nlimit 2,10\r\nlimt 3,10\r\n```\r\n\r\njava code\r\n```\r\n//junit test java code \r\n        Pageable pageable1 = new PageRequest(1,10);\r\n        List<UserPo> pos1 = userRepository.getWithoutUserIdListBy("appcode_idxyer_android", new Date(),pageable1);\r\n        log.info("pos1:{}",pos1.toString());\r\n\r\n        Pageable pageable2 = new PageRequest(2,10);\r\n        List<UserPo> pos2 = userRepository.getWithoutUserIdListBy("appcode_idxyer_android", new Date(),pageable2);\r\n        log.info("pos2:{}",pos2.toString());\r\n\r\n        Pageable pageable3 = new PageRequest(3,10);\r\n        List<UserPo> pos3 = userRepository.getWithoutUserIdListBy("appcode_idxyer_android", new Date(),pageable3);\r\n        log.info("pos3:{}",pos3.toString());\r\n\r\n//orm code\r\n  @Query("SELECT u FROM UserPo u WHERE u

In [28]:
print(test_sample.loc[1313,'original_issue_body'])

version：4.0.0.M1-SNAPSHOT
branch：dev 
orm: spring jpa

not sharding table use default datasource，but  page data is the same，for the following limit:
```
limit 1,10
limit 2,10
limt 3,10
```

java code
```
//junit test java code 
        Pageable pageable1 = new PageRequest(1,10);
        List<UserPo> pos1 = userRepository.getWithoutUserIdListBy("appcode_idxyer_android", new Date(),pageable1);
        log.info("pos1:{}",pos1.toString());

        Pageable pageable2 = new PageRequest(2,10);
        List<UserPo> pos2 = userRepository.getWithoutUserIdListBy("appcode_idxyer_android", new Date(),pageable2);
        log.info("pos2:{}",pos2.toString());

        Pageable pageable3 = new PageRequest(3,10);
        List<UserPo> pos3 = userRepository.getWithoutUserIdListBy("appcode_idxyer_android", new Date(),pageable3);
        log.info("pos3:{}",pos3.toString());

//orm code
  @Query("SELECT u FROM UserPo u WHERE u.userId is NULL and u.appName = ?1 and u.modifyTime < ?2")
  List<UserPo> getWitho

In [29]:
test_sample.loc[1313,'llama3_70b_body_summary_check'] = 'seems to be the model provided recommendations'

In [30]:
test_sample.loc[1323,'llama3_70b_body_summary']

"It seems like you're experiencing issues with the SkyWalking agent, , specifically with the `ProtectiveShieldMatcher` and `InstanceMethodsAroundInterceptor` classes.\n\nTo summarize, you've tried to debug the issue and found that:\n\n1. The `AgentClassLoader` is not loading the `skywalking-agent.jar` package.\n2. When you manually add the `skywalking-agent.jar` to the `agent/plugin/` directory, the `InstanceMethodsAroundInterceptor` class is loaded, but throws a `CastException`.\n\nYou're looking for help or suggestions on how to resolve this issue.\n\nCan you please provide more information about your environment, such as:\n\n* SkyWalking agent version\n* Java version\n* Application server/container (e.g. Tomcat, Jetty, etc.)\n* Any custom configurations or plugins used with the SkyWalking agent\n\nThis will help me better understand the context and provide more accurate guidance."

In [31]:
test_sample.loc[1323,'llama3_70b_body_summary_corrected'] = "To summarize, you've tried to debug the issue and found that:\n\n1. The `AgentClassLoader` is not loading the `skywalking-agent.jar` package.\n2. When you manually add the `skywalking-agent.jar` to the `agent/plugin/` directory, the `InstanceMethodsAroundInterceptor` class is loaded, but throws a `CastException`.\n\nYou're looking for help or suggestions on how to resolve this issue."

In [32]:
test_sample.loc[1323,'llama3_70b_body_summary_check'] = 'first sentence and the last part are removed'

In [33]:
test_sample.loc[1424,'llama3_70b_body_summary']

'This is a Java stack trace from an Elasticsearch application. It appears to be related to authentication and security features in Elasticsearch.\n\nThe stack trace shows the sequence of method calls that led to an error or exception being thrown. The methods are from various classes in the Elasticsearch codebase, , including `org.elasticsearch.action`, `org.elasticsearch.xpack.security.authc`, `org.elasticsearch.http.netty4`, and others.\n\nThe error message itself is not included in this stack trace,<|begin_of_text|>2019-12-17 14:34:05,345 [INFO ][o.e.h.n.Netty4HttpServerTransport] [node-name] publish_address {ip-address:9200}, bound_addresses {[::]:9200}'

In [34]:
test_sample.loc[1424,'llama3_70b_body_summary_check'] = 'OK'

In [35]:
test_sample.loc[1557,'llama3_70b_body_summary']

"This appears to be a thread dump from a Java application that's using Selenium WebDriver. The dump shows multiple threads waiting on conditions, which suggests that the application is experiencing some kind of deadlock or synchronization issue.\n\nHere are some observations and potential areas for investigation:\n\n1. **ThreadPoolExecutor**: Multiple threads are waiting on `SynchronousQueue$TransferStack` objects, which are used by `ThreadPoolExecutor` to manage task execution. This could indicate that the thread pool is not properly configured or that there's a deadlock between threads.\n2. **ForkJoinPool**: Several threads are waiting on `ForkJoinPool` objects, which are used for parallel processing. This might suggest that the application is using parallelism, but it's not clear why these threads are blocked.\n3. **nioEventLoopGroup**: One thread is running and holding locks on `SelectedSelectionKeySet` and `EPollSelectorImpl` objects, which are related to Netty's NIO event loop. T

In [36]:
test_sample.loc[1557,'llama3_70b_body_summary_check'] = 'seems to be the model provided recommendations'

In [37]:
test_sample.loc[1805,'llama3_70b_body_summary']

"This is an error log from Apache SkyWalking, a distributed tracing system. The error occurs during the startup process of the OAP (Observability Analysis Platform) server.\n\nThe main error message is:\n```\ncom.linecorp.armeria.client.UnprocessedRequestException: com.linecorp.armeria.client.endpoint.EmptyEndpointGroupException: Unable to select endpoints from: HealthCheckedEndpointGroup{endpoints=[], numEndpoints=0, candidates=[Endpoint{es8:9200, weight=1000}], numCandidates=1, selectionStrategy=class com.linecorp.armeria.client.endpoint.WeightedRoundRobinStrategy, initialized=true}\n```\nThis error is caused by an `EmptyEndpointGroupException`, which means that the Armeria client (a Java library for building scalable and fault-tolerant HTTP clients) cannot select any endpoints from the `HealthCheckedEndpointGroup`.\n\nThe log also shows a stack trace of the error, including calls to Netty (a Java network programming framework) and SkyWalking's internal classes.\n\nThe user has provi

In [38]:
test_sample.loc[1805,'llama3_70b_body_summary_check'] = 'OK'

In [39]:
test_sample.loc[2022,'llama3_70b_body_summary']

'Here is a summary of the log messages:\n\n* The system is shutting down and deregistering services from Nacos.\n* There are repeated errors sending requests to Nacos with the error message "Client not connected, current status: SHUTDOWN".\n* The requests are being retried up to 2 times before giving up.\n* The services being deregistered include `com.jrmall.system.auth.menu.api.SysAuthMenuApi` and `com.jrmall.system.data.opt.api.SysGroupOptApi`.\n* The instance information includes IP address `172.16.85.134`, port `0`, weight `1.0`, healthy `true`, enabled `true`, ephemeral `true`, cluster name `DEFAULT`, service name `null`, and metadata `{}`.'

In [40]:
test_sample.loc[2022,'llama3_70b_body_summary_check'] = 'OK'

In [41]:
test_sample.loc[2172,'llama3_70b_body_summary']

'This is a Java crash log that provides detailed information about the system, JVM, and process that crashed. Here\'s a summary of the contents:\n\n**System Information**\n\n* OS: CentOS release 6.5 (Final)\n* Architecture: x86_64\n* Processor: Intel(R) Xeon(R) CPU E5-2620 0 @ 2.00GHz (8 cores, 16 threads)\n* Memory: 3973620 kB (52500 kB free), swap 4112376 kB (3429676 kB free)\n\n**JVM Information**\n\n* JVM version: Java HotSpot(TM) 64-Bit Server VM (25.51-b03)\n* JRE version: 1.8.0_51-b16\n* Built on: Jun  8 2015 19:28:07 by "java_re" with gcc 4.3.0 20080428 (Red Hat 4.3.0-8)\n\n**Process Information**\n\n* Command line arguments: org.jruby.Main --1.9 /data0/logstash/app/logstash-2.0.0/lib/bootstrap/environment.rb logstash/runner.rb agent -f /data0/logstash/app/logstash-2.0.0/conf -l /data0/logstash/app/logstash-2.0.0/logs/logstash.log\n* VM arguments: various options including garbage collection, heap size, and classpath settings\n\n**Memory Information**\n\n* Memory usage: 3371800

In [42]:
test_sample.loc[2172,'llama3_70b_body_summary_check'] = 'OK'

In [43]:
test_sample.loc[2469,'llama3_70b_body_summary']

"This is a log output from an Apache Pulsar client application. It appears to be a test or debug output, as it includes detailed information about the Pulsar client's configuration and behavior.\n\nHere's a breakdown of the log:\n\n1. The first section shows the Pulsar client connecting to a broker at `localhost:14002` through a proxy.\n2. The client creates a producer and consumer for a topic named `persistent://public/default/redeliver-topic-partition-0`.\n3. The consumer subscribes to the topic and starts receiving messages.\n4. The log shows several warnings about unacked messages timing out, indicating that the consumer is not acknowledging messages quickly enough.\n5. At around 09:00:48, the client creates a new producer for a dead-letter queue (DLQ) topic named `persistent://public/default/redeliver-topic-partition-0-failing-subscription-DLQ`.\n6. The log shows several INFO-level messages about the client closing producers and consumers.\n7. Finally, the log ends with a WARN-lev

In [44]:
test_sample.loc[2469,'llama3_70b_body_summary_check'] = 'OK'

In [45]:
test_sample.loc[2615,'llama3_70b_body_summary']

'Here is a summary of the error message:\n\nThe build failed due to an error in the Maven project. The error is related to the `P2ArtifactRepositoryLayout` class, which is not present.\n\nThe specific errors are:\n\n* `java.lang.TypeNotPresentException: Type org.eclipse.tycho.core.p2.P2ArtifactRepositoryLayout not present`\n* `java.lang.UnsupportedClassVersionError: org/eclipse/tycho/core/p2/P2ArtifactRepositoryLayout has been compiled by a more recent version of the Java Runtime (class file version 55.0), this version of the Java Runtime only recognizes class file versions up to 52.0`\n\nThe error occurs while trying to build the project `org.jkiss.dbeaver:dbeaver:1.0.0-SNAPSHOT`.'

In [46]:
test_sample.loc[2615,'llama3_70b_body_summary_check'] = 'OK'

In [47]:
test_sample.loc[2624,'llama3_70b_body_summary']

"This is a configuration file for ThingsBoard, an open-source IoT platform. The file defines various settings and properties for the platform's components, including:\n\n1. License: specifies the license secret obtained from the ThingsBoard License Portal.\n2. Instance data: defines the instance data file path and other related settings.\n3. Swagger: configures the Swagger API documentation, including title, description, contact information, and license details.\n4. Queue: sets up the message queue system, including the type of queue (in-memory, Kafka, AWS SQS, PubSub, RabbitMQ), and various properties for each type.\n5. Partitions: defines the partitioning strategy for the queue.\n6. Transport API: configures the transport API, including request and response topics, max pending requests, and poll intervals.\n7. Core: sets up the core component, including topic, poll interval, partitions, and pack processing timeout.\n8. JS Executor: configures the JavaScript executor, including reques

In [48]:
test_sample.loc[2624,'llama3_70b_body_summary_check'] = 'OK'

In [49]:
test_sample.loc[2759,'llama3_70b_body_summary']

"Here is a summary of the log file:\n\n**ThingsBoard Engine Installation**\n\nThe log file shows that the ThingsBoard engine is being installed and configured.\n\n**Kafka Consumer Configuration**\n\nThe log file shows that multiple Kafka consumers are being created for different topics, including `tb_rule_engine.sq` with partitions 0-8. Each consumer is subscribing to a specific topic and partition.\n\n**Error Occurred**\n\nAn error occurred during the application startup, causing the application to fail. The error message indicates a `RuntimeException` with an underlying `ExecutionException` caused by a `TimeoutException`. This suggests that there was a timeout while waiting for a response from a request.\n\n**Config File**\n\nThe config file shows various settings and configurations for the ThingsBoard engine, including:\n\n* Java options (e.g., GC logging, heap dump on OOM)\n* Database settings (e.g., PostgreSQL connection details)\n* Queue settings (e.g., Kafka server address)\n* S

In [50]:
test_sample.loc[2759,'llama3_70b_body_summary_corrected'] = "**ThingsBoard Engine Installation**\n\nThe log file shows that the ThingsBoard engine is being installed and configured.\n\n**Kafka Consumer Configuration**\n\nThe log file shows that multiple Kafka consumers are being created for different topics, including `tb_rule_engine.sq` with partitions 0-8. Each consumer is subscribing to a specific topic and partition.\n\n**Error Occurred**\n\nAn error occurred during the application startup, causing the application to fail. The error message indicates a `RuntimeException` with an underlying `ExecutionException` caused by a `TimeoutException`. This suggests that there was a timeout while waiting for a response from a request.\n\n**Config File**\n\nThe config file shows various settings and configurations for the ThingsBoard engine, including:\n\n* Java options (e.g., GC logging, heap dump on OOM)\n* Database settings (e.g., PostgreSQL connection details)\n* Queue settings (e.g., Kafka server address)\n* SQL data folder location"

In [51]:
test_sample.loc[2759,'llama3_70b_body_summary_check'] = 'first and last sentences are removed'

In [52]:
test_sample.loc[3021,'llama3_70b_body_summary']

"This is a logcat output showing two instances of a fatal exception occurring in an Android app. The exceptions are caused by the inability to instantiate a fragment class, specifically `org.schabi.newpipe.fragments.EmptyFragment`.\n\nHere's a breakdown of the error:\n\n1. The first instance occurs at 14:05:23 and has a process ID of 19008.\n2. The second instance occurs at 14:06:10 and has a process ID of 19292.\n\nThe error message is:\n```\njava.lang.RuntimeException: Unable to start activity ComponentInfo{org.schabi.newpipe/org.schabi.newpipe.MainActivity}: androidx.fragment.app.Fragment$InstantiationException: Unable to instantiate fragment org.schabi.newpipe.fragments.EmptyFragment: could not find Fragment constructor\n```\n\nThis indicates that the `EmptyFragment` class does not have a no-argument constructor, which is required for fragments.\n\nThe error is caused by the following line:\n```\nat androidx.fragment.app.Fragment.instantiate(Fragment.java:563)\n```\n\nThis suggests

In [53]:
test_sample.loc[3021,'original_issue_body']

'Hello.\r\n\r\n### Checklist\r\n<!-- This checklist is COMPULSORY. The first box has been checked for you to show you how it is done. -->\r\n\r\n- [x] I am using the latest version - v0.21.3 <!-- Check https://github.com/TeamNewPipe/NewPipe/releases -->\r\n- [x] I checked, but didn\'t find any duplicates (open OR closed) of this issue in the repo. <!-- Seriously, check. O_O -->\r\n- [x] I have read the contribution guidelines given at https://github.com/TeamNewPipe/NewPipe/blob/HEAD/.github/CONTRIBUTING.md.\r\n- [x] This issue contains only one bug. I will open one issue for every bug report I want to file.\r\n\r\n### Steps to reproduce the bug\r\n1. Open NewPipe\r\n2. Click on the video (sometimes on a video that requires confirmation of age 18+)\r\n\r\n\r\n### Actual behaviour\r\n<!-- Tell us what happens with the steps given above. -->\r\nI\'ve been experiencing this problem for quite some time, for about two months now.\r\nOn versions: v0.21.2, v0.21.3. I don\'t know on previous on

In [54]:
test_sample.loc[3021,'llama3_70b_body_summary_corrected'] = "Here's a breakdown of the error:\n\n1. The first instance occurs at 14:05:23 and has a process ID of 19008.\n2. The second instance occurs at 14:06:10 and has a process ID of 19292.\n\nThe error message is:\n```\njava.lang.RuntimeException: Unable to start activity ComponentInfo{org.schabi.newpipe/org.schabi.newpipe.MainActivity}: androidx.fragment.app.Fragment$InstantiationException: Unable to instantiate fragment org.schabi.newpipe.fragments.EmptyFragment: could not find Fragment constructor"

In [55]:
test_sample.loc[3021,'llama3_70b_body_summary_check'] = 'interpretation of the model is removed'

In [56]:
test_sample.loc[3027,'llama3_70b_body_summary']

'Here is a summary of the log output:\n\n* There are multiple errors occurring due to tables not existing in the database.\n* The specific tables mentioned are `swtest.service_inventory` and `swtest.service_instance_inventory`.\n* These errors are being thrown by various components of the Apache SkyWalking system, including the storage plugin, cache, and receiver.\n* The errors are causing issues with service instance heartbeats, metric reporting, and other functionality.\n\nThe log output does not provide a clear cause for why these tables do not exist, but it suggests that there may be an issue with the database configuration or setup.'

In [57]:
test_sample.loc[3027,'llama3_70b_body_summary_check'] = 'OK'

In [58]:
test_sample.loc[3045,'llama3_70b_body_summary']

'Here is a summary of the crash logs:\n\nAll 19 crash logs have the same error message and stack trace, indicating that they are related to the same issue.\n\nThe error message is `org.schabi.newpipe.local.feed.service.FeedLoadService$RequestException`, which is caused by a `ParsingException` with the message "Could not get channel id".\n\nThe exception occurs in the `YoutubeChannelExtractor.getId()` method, which suggests that there is an issue with parsing the YouTube channel ID.\n\nThe stack trace shows that the error occurs when trying to load a feed from a specific YouTube channel URL. The URLs are different for each crash log, but they all follow the same format (`https://www.youtube.com/channel/<channel_id>`).\n\nThere is no device-specific information provided in the crash logs, so it\'s unclear if this issue is related to a specific Android version or device model.'

In [59]:
test_sample.loc[3045,'llama3_70b_body_summary_check'] = 'OK'

In [60]:
test_sample.loc[3103,'llama3_70b_body_summary']

"The output shows a list of errors encountered during the compilation of Java code. The errors are related to unresolved types and methods in various classes, including:\n\n* `JexlEngine`, `JexlContext`, `Expression`, and `JexlException` from the Apache Commons JEXL library\n* `PlainSelect` and `Update` from a SQL parsing library\n\nThe errors indicate that the compiler is unable to find these types and methods, suggesting that there may be issues with the project's dependencies or imports.\n\nThere are 72 errors in total, and the output provides information on how to troubleshoot and resolve the issues."

In [61]:
test_sample.loc[3103,'llama3_70b_body_summary_check'] = 'OK'

In [62]:
test_sample.loc[3448,'llama3_70b_body_summary']

"This is a log output from an Elasticsearch node, showing various events and errors that occurred during startup. Here's a breakdown of the log:\n\n**Loaded modules**: The first several lines show that various X-Pack modules were loaded successfully.\n\n**Warnings**: There are two warnings related to Java 19 and MMapDirectory.\n\n**Node configuration**: The log shows the node name, ID, cluster name, and roles.\n\n**Security**: Security is enabled, and some security-related settings are configured.\n\n**NettyAllocator**: A NettyAllocator is created with specific configurations.\n\n**Rate limit**: A rate limit of 40mb is set for the node.\n\n**Discovery**: The discovery type is set to multi-node, and seed hosts providers are configured.\n\n**Initialization**: The node is initialized, and various components are started.\n\n**Bootstrap checks failure**: However, the bootstrap checks fail due to a low value for `vm.max_map_count` (65530), which is too low. This causes Elasticsearch to exit 

In [63]:
test_sample.loc[3448,'llama3_70b_body_summary_check'] = 'OK'

In [64]:
test_sample.loc[3891,'llama3_70b_body_summary']

"This appears to be a crash log from an iOS app built with React Native. The log shows the call stacks for 33 threads, including the main thread and various worker threads.\n\nHere's a brief summary of each thread:\n\n**Threads 1-18**: These threads are all related to GCD (Grand Central Dispatch) and are likely handling asynchronous tasks such as networking requests or database operations.\n\n**Thread 19**: This thread is running an AVAudioSession notify thread, which is used for audio processing.\n\n**Threads 20-24**: These threads are all WTF::AutomaticThread instances, which are used by the JavaScriptCore framework to run JavaScript code in separate threads.\n\n**Thread 25**: This thread is running a com.facebook.react.JavaScript thread, which is responsible for executing JavaScript code in the React Native app.\n\n**Thread 26**: This thread is a worker thread created by libdispatch.dylib, which is used for asynchronous task execution.\n\n**Thread 27**: This thread is another WTF::A

In [65]:
test_sample.loc[3891,'llama3_70b_body_summary_check'] = 'OK'

In [66]:
test_sample.loc[4221,'llama3_70b_body_summary']

"The infamous Swift version mismatch error!\n\nThis error occurs when a module (in this case, `Charts`) is compiled with a different version of Swift than the one used by your project.\n\nHere are some potential solutions to try:\n\n1. **Update `Charts` to use Swift 5.1.2**:\nCheck if there's an updated version of `Charts` that supports Swift 5.1.2. You can do this by running `pod update Charts` in your terminal.\n2. **Downgrade your Swift version**:\nIf you're using a newer version of Xcode, try downgrading to an earlier version that uses Swift 5.1. This might not be feasible if you need features from the latest Xcode version.\n3. **Use a Swift version manager**:\nTools like `swiftversion` or `swiftenv` allow you to manage multiple Swift versions on your system. You can use these tools to switch between Swift 5.1 and 5.1.2.\n4. **Exclude arm64 architecture**:\nAs a temporary workaround, you can try excluding the arm64 architecture from your build settings. This might not be ideal, but

In [67]:
test_sample.loc[4221,'llama3_70b_body_summary_check'] = 'seems to be the model provided recommendations'

In [68]:
test_sample.loc[4468,'llama3_70b_body_summary']

'This is a configuration file for the DBeaver database management system. It contains various settings and preferences for the application, including:\n\n* UI settings: font sizes, colors, and layout options\n* Editor settings: syntax highlighting, auto-completion, and formatting options\n* Database connection settings: driver configurations, authentication details, and query execution options\n* Data format settings: date, time, and number formatting options\n* Search and filtering settings: search metadata, object types, and result set limits\n* Miscellaneous settings: animation, window management, and other application behaviors\n\nThe file is organized into sections, each starting with a `/` character, followed by the name of the section. Within each section, there are key-value pairs that specify the setting or preference.\n\nFor example, the line `/bundle_defaults/org.jkiss.dbeaver.core/dataformat.type.number.maxFractionDigits=3` sets the maximum number of fraction digits for num

In [69]:
test_sample.loc[4468,'llama3_70b_body_summary_check'] = 'OK'

In [70]:
test_sample.loc[4654,'llama3_70b_body_summary']

"This is a log output from a Spring Boot application that uses Spring Cloud and other dependencies. Here's a breakdown of the log:\n\n**Initial Startup**\n\n* The application starts up, and various beans are registered with JMX (Java Management Extensions) for monitoring.\n* Beans such as `environmentManager`, `restartEndpoint`, `refreshScope`, and others are autodetected and registered with JMX.\n\n**Error during Startup**\n\n* An error occurs during startup due to an `InstanceAlreadyExistsException` when trying to register the `EnvironmentManager` bean with JMX. This causes the application startup to fail.\n* The error is caused by a nested exception, which is also an `InstanceAlreadyExistsException`.\n\n**Subsequent Log Output**\n\n* Despite the initial error, the application continues to log various events and bean registrations.\n* Beans such as `configurationPropertiesRebinder`, `refreshEndpoint`, and others are registered with JMX.\n* The application starts up various components

In [71]:
test_sample.loc[4654,'llama3_70b_body_summary_check'] = 'OK'

In [72]:
test_sample.loc[4892,'llama3_70b_body_summary']

"This appears to be a configuration file for an Apache Pulsar cluster, along with some log output from a BookKeeper instance.\n\nHere's a breakdown of the contents:\n\n**Configuration File**\n\nThe first part of the output is a JSON object that represents the configuration for an Apache Pulsar cluster. It contains various settings and properties related to topics, subscriptions, brokers, bookies, ZooKeeper, and more.\n\nSome notable settings include:\n\n* `brokerServicePort`: The port number used by the broker service (6650).\n* `loadBalancerReportUpdateMaxIntervalMinutes`: The maximum interval in minutes for updating load balancer reports (15).\n* `managedLedgerNumSchedulerThreads`: The number of threads used for scheduling managed ledgers (4).\n* `tlsEnabled`: Whether TLS is enabled (false).\n\n**Log Output**\n\nThe second part of the output appears to be log messages from a BookKeeper instance. BookKeeper is a distributed log storage system that's often used in conjunction with Apac

In [73]:
test_sample.loc[4892,'llama3_70b_body_summary_check'] = 'OK'

In [74]:
test_sample.loc[4897,'llama3_70b_body_summary']

'This is an Eclipse error log, which indicates that there are several bundles (plugins) that cannot be resolved or started due to missing dependencies. Here\'s a summary of the errors:\n\n1. `org.eclipse.e4.ui.workbench.renderers.swt`:\n\t* Unresolved requirement: `Require-Bundle: org.eclipse.e4.core.services; bundle-version="0.9.0"`\n\t* Unresolved requirement: `Import-Package: org.osgi.service.component.annotations; version="1.2.0"; resolution:="optional"`\n2. `org.eclipse.e4.ui.workbench.swt`:\n\t* Unresolved requirement: `Import-Package: org.osgi.service.component.annotations; version="1.2.0"; resolution:="optional"`\n3. `org.eclipse.equinox.p2.discovery.compatibility`:\n\t* Unresolved requirement: `Require-Bundle: org.eclipse.core.runtime; bundle-version="3.3.0"`\n4. `org.eclipse.equinox.p2.transport.ecf`:\n\t* Unresolved requirement: `Require-Bundle: org.eclipse.ecf; bundle-version="3.1.0"`\n5. `org.eclipse.equinox.p2.ui`:\n\t* Unresolved requirement: `Require-Bundle: org.eclipse

In [75]:
test_sample.loc[4897,'llama3_70b_body_summary_corrected'] = """1. `org.eclipse.e4.ui.workbench.renderers.swt`:\n\t* Unresolved requirement: `Require-Bundle: org.eclipse.e4.core.services; bundle-version="0.9.0"`\n\t* Unresolved requirement: `Import-Package: org.osgi.service.component.annotations; version="1.2.0"; resolution:="optional"`\n2. `org.eclipse.e4.ui.workbench.swt`:\n\t* Unresolved requirement: `Import-Package: org.osgi.service.component.annotations; version="1.2.0"; resolution:="optional"`\n3. `org.eclipse.equinox.p2.discovery.compatibility`:\n\t* Unresolved requirement: `Require-Bundle: org.eclipse.core.runtime; bundle-version="3.3.0"`\n4. `org.eclipse.equinox.p2.transport.ecf`:\n\t* Unresolved requirement: `Require-Bundle: org.eclipse.ecf; bundle-version="3.1.0"`\n5. `org.eclipse.equinox.p2.ui`:\n\t* Unresolved requirement: `Require-Bundle: org.eclipse.core.runtime; bundle-version="[3.18.0,4.0.0)"`\n\t* Unresolved requirement: `Require-Bundle: org.eclipse.ui; bundle-version="3.107.0"`\n6. `org.eclipse.equinox.p2.ui.importexport`:\n\t* Unresolved requirement: `Import-Package: org.eclipse.equinox.internal.p2.ui`\n7. `org.jkiss.dbeaver.ext.h2gis`:\n\t* Unresolved requirement: `Require-Bundle: org.jkiss.dbeaver.model`\n8. `org.eclipse.e4.ui.css.core`:\n\t* Unresolved requirement: `Require-Bundle: org.apache.batik.css; bundle-version="[1.9.1,2.0.0)"`\n9. `org.eclipse.jsch.ui`:\n\t* Unresolved requirement: `Require-Bundle: org.eclipse.ui; bundle-version="[3.5.0,4.0.0)"""

In [76]:
test_sample.loc[4897,'llama3_70b_body_summary_check'] = 'first and last parts are removed'

In [77]:
test_sample.loc[4932,'llama3_70b_body_summary']

"This is a Java stack trace from an Eclipse-based application,<|begin_of_text|>2014. It appears to be a complex error message with multiple exceptions and errors. Here's a breakdown of the main issues:\n\n1. `java.lang.NoClassDefFoundError`: The class `org/eclipse/jface/resource/ResourceLocator` cannot be found.\n2. `org.eclipse.swt.SWTException: Widget is disposed`: A widget has been disposed, , but an attempt was made to access or manipulate it.\n\nThe errors occur in the context of a wizard dialog (e.g., `NewConnectionDialog`) and involve various Eclipse plugins and components, such as:\n\n* `org.jkiss.dbeaver` (a database management tool)\n* `org.eclipse.jface` (Eclipse's Java-based UI framework)\n* `org.eclipse.swt` (Eclipse's Standard Widget Toolkit)\n\nThe stack trace suggests that the error occurs when trying to create a new connection in the DBeaver application. The exact cause of the issue is unclear, but it may be related to:\n\n* A missing or incompatible plugin or library\

In [78]:
test_sample.loc[4932,'llama3_70b_body_summary_corrected'] = """1. `java.lang.NoClassDefFoundError`: The class `org/eclipse/jface/resource/ResourceLocator` cannot be found.\n2. `org.eclipse.swt.SWTException: Widget is disposed`: A widget has been disposed, , but an attempt was made to access or manipulate it.\n\nThe errors occur in the context of a wizard dialog (e.g., `NewConnectionDialog`) and involve various Eclipse plugins and components, such as:\n\n* `org.jkiss.dbeaver` (a database management tool)\n* `org.eclipse.jface` (Eclipse's Java-based UI framework)\n* `org.eclipse.swt` (Eclipse's Standard Widget Toolkit)\n\nThe stack trace suggests that the error occurs when trying to create a new connection in the DBeaver application. The exact cause of the issue is unclear, but it may be related to:\n\n* A missing or incompatible plugin or library\n* An incorrect configuration or setup\n* A bug in one of the involved components"""

In [79]:
test_sample.loc[4932,'llama3_70b_body_summary_check'] = 'first and last parts are removed'

Running LLaMA classification prompt again

In [80]:
with tqdm(total=test_sample.shape[0], ncols=100) as pbar:
    for idx in test_sample.index:
        try:
            test_sample.loc[idx, 'llama3_70b_prediction_round2'] = llm.invoke(f"""Classify the following issue as "bug", "enhancement", "question" or "other" and rate your classification confidence and issue understandability in range 0 - 100. Your answer should only include:
Classification: your classification answer
Prediction Confidence: your level of confidence
Issue Understandability: level of understandability
Do not provide any explanation.
Issue Title: {test_sample.loc[idx,'original_issue_title']}
Issue Body: {test_sample.loc[idx,'llama3_70b_body_summary_corrected']}
""")
            pbar.update(1)
        except Exception as e:
            print(f'Exception at index {idx}: {e}')
            continue

print(f'Categorization of {test_sample.shape[0]} issues is complete')

100%|███████████████████████████████████████████████████████████████| 28/28 [01:49<00:00,  3.89s/it]

Categorization of 28 issues is complete


In [81]:
test_sample['llama3_70b_prediction_round2'].value_counts()

,count
llama3_70b_prediction_round2,
Classification: bug\nPrediction Confidence: 90\nIssue Understandability: 80,10
Classification: bug\nPrediction Confidence: 90\nIssue Understandability: 95,6
Classification: bug\nPrediction Confidence: 100\nIssue Understandability: 90,5
Classification: bug\nPrediction Confidence: 95\nIssue Understandability: 90,3
Classification: other\nPrediction Confidence: 80\nIssue Understandability: 60,2
Classification: question\nPrediction Confidence: 90\nIssue Understandability: 95,1
Classification: bug\nPrediction Confidence: 80\nIssue Understandability: 90,1


In [82]:
test_sample.to_pickle('msc_test_sample_summarized_llama3_70b_preds.pkl')